In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 添加项目根目录到路径（notebook 版本）
# 方法1: 如果 notebook 在 notebooks/ 目录下，向上查找父目录
ROOT_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
# 方法2（备选）: 基于当前工作目录，假设在项目根目录运行
# ROOT_DIR = Path.cwd()
sys.path.append(str(ROOT_DIR))

from src import (
    theory, 
    SDEConfig, run_sde_simulation, 
    NetworkConfig, NetworkAgentModel,
    calculate_autocorrelation
)

# 设置绘图风格
plt.style.use('seaborn-v0_8-paper')
sns.set_palette("husl")
OUTPUT_DIR = ROOT_DIR / "final_figures"
OUTPUT_DIR.mkdir(exist_ok=True)

def experiment_1_bifurcation_diagram():
    """
    图 1: 理论相图与 SDE 验证 (Phase Diagram)
    展示随着 r 增加，系统如何从单稳态(q=0)分岔到双稳态(q!=0)。
    """
    print("Running Experiment 1: Bifurcation Diagram...")
    
    # 参数设置
    phi, theta, k_avg = 0.6, 0.4, 50
    n_m, n_w = 10, 5
    
    # 1. 理论计算
    chi = theory.calculate_chi(phi, theta, k_avg)
    rc_theory = theory.calculate_rc(n_m, n_w, chi)
    print(f"  -> Theoretical rc = {rc_theory:.4f}")
    
    r_theory = np.linspace(0, 1.0, 200)
    q_theory = []
    
    # 解析解：q = sqrt(|alpha|/u) if r > rc else 0
    alpha_arr, u = theory.get_gl_params(r_theory, rc_theory)
    for a_val in alpha_arr:
        if a_val < 0: # r > rc, alpha < 0
            q_val = np.sqrt(abs(a_val) / u)
        else:
            q_val = 0
        q_theory.append(q_val)
        
    # 2. SDE 模拟验证点
    r_sim = np.linspace(0, 1.0, 15)
    q_sim_mean = []
    q_sim_std = []
    
    for r in tqdm(r_sim, desc="  SDE Simulation"):
        alpha, _ = theory.get_gl_params(r, rc_theory)
        # 如果 r > rc，加入一点初始扰动防止卡在 0
        q0 = 0.1 if r > rc_theory else 0.0
        cfg = SDEConfig(alpha=float(alpha), u=u, sigma=0.05, steps=5000, dt=0.05)
        _, traj = run_sde_simulation(cfg, q0=q0)
        # 取最后 20% 的轨迹算稳态
        steady_state = traj[1000:]  # keep sign to show symmetric branches 
        q_sim_mean.append(np.mean(steady_state))
        q_sim_std.append(np.std(steady_state))

    # 3. 绘图
    fig, ax = plt.subplots(figsize=(8, 6))
    # 理论线
    ax.plot(r_theory, q_theory, 'k--', label='Theoretical Prediction (Mean Field)', linewidth=2)
    ax.plot(r_theory, [-q for q in q_theory], 'k--', linewidth=2)
    
    # 模拟点
    ax.errorbar(r_sim, q_sim_mean, yerr=q_sim_std, fmt='o', color='crimson', 
                label='SDE Simulation', capsize=3, alpha=0.8)
    
    ax.axvline(rc_theory, color='blue', linestyle=':', label=f'Critical $r_c$ ({rc_theory:.2f})')
    ax.set_xlabel('Control Parameter $r$ (Mainstream Removal)')
    ax.set_ylabel('Polarization Order Parameter $q$')
    ax.set_title(rf'Bifurcation Diagram (Sensitivity $\chi={chi:.2f}$)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.savefig(OUTPUT_DIR / "Fig1_Bifurcation.png", dpi=300)
    plt.close()
    print("  -> Saved Fig1")

def experiment_2_potential_landscape():
    """
    图 2: 势能地貌演化 (Potential Landscape)
    展示 V(q) 如何从单底碗变为双底碗。
    """
    print("Running Experiment 2: Potential Landscape...")
    
    phi, theta, k_avg = 0.6, 0.4, 50
    n_m, n_w = 10, 5
    chi = theory.calculate_chi(phi, theta, k_avg)
    rc = theory.calculate_rc(n_m, n_w, chi)
    
    # 选取三个代表性的 r
    r_values = [rc - 0.2, rc, rc + 0.2]
    labels = ['Stable ($r < r_c$)', 'Critical ($r \\approx r_c$)', 'Polarized ($r > r_c$)']
    colors = ['green', 'orange', 'red']
    
    q_range = np.linspace(-2, 2, 200)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    
    for r, label, color in zip(r_values, labels, colors):
        alpha, u = theory.get_gl_params(r, rc)
        V = theory.potential_energy(q_range, alpha, u)
        ax.plot(q_range, V, label=f'{label}, r={r:.2f}', color=color, linewidth=2.5)
        
    ax.set_xlabel('Polarization $q$')
    ax.set_ylabel('Effective Potential $V_{eff}(q)$')
    ax.set_title('Evolution of the Potential Landscape')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.5, 1.0) # 限制y轴以便看清碗底
    
    plt.savefig(OUTPUT_DIR / "Fig2_Potential.png", dpi=300)
    plt.close()
    print("  -> Saved Fig2")

def experiment_3_network_robustness():
    """
    图 3: 网络拓扑效应验证 (Network Robustness)
    对比理论曲线与真实网络模拟的结果，展示拓扑结构带来的偏移。
    """
    print("Running Experiment 3: Network Simulation...")
    
    # 理论基准
    phi, theta, k_avg = 0.6, 0.4, 6 # 注意：为了模拟跑得动，k_avg 设小一点，或者网络设小一点
    n_m, n_w = 10, 5
    chi = theory.calculate_chi(phi, theta, k_avg)
    rc_theory = theory.calculate_rc(n_m, n_w, chi)
    
    # 网络参数
    N_NODES = 500
    beta = 0.0 # 先做无邻居耦合的基准验证
    
    r_scan = np.linspace(0, 1.0, 11)
    q_net_mean = []
    q_net_std = []
    
    for r in tqdm(r_scan, desc="  Network ABM"):
        # 跑多次取平均以消除随机性
        q_trials = []
        for seed in range(3):
            cfg = NetworkConfig(
                n=N_NODES, avg_degree=k_avg, model="ba", beta=beta,
                r=r, n_m=n_m, n_w=n_w, phi=phi, theta=theta, seed=seed
            )
            model = NetworkAgentModel(cfg)
            _, q_traj, _ = model.run(steps=200, record_interval=10)
            q_trials.append(np.mean(q_traj[-5:])) # 取最后几步的绝对值平均
            
        q_net_mean.append(np.mean(q_trials))
        q_net_std.append(np.std(q_trials))
        
    # 绘图
    fig, ax = plt.subplots(figsize=(8, 6))
    
    # 理论线 (作为参考背景)
    alpha_arr, u = theory.get_gl_params(np.linspace(0,1,100), rc_theory)
    q_th = [np.sqrt(abs(a)) if a<0 else 0 for a in alpha_arr]
    ax.plot(np.linspace(0,1,100), q_th, 'k--', alpha=0.3, label='Mean-Field Theory')
    
    # 网络点
    ax.errorbar(r_scan, q_net_mean, yerr=q_net_std, fmt='s-', color='dodgerblue',
                label=f'Network ABM (BA, N={N_NODES})', capsize=3)
    
    ax.axvline(rc_theory, color='gray', linestyle=':', alpha=0.5)
    ax.set_xlabel('Control Parameter $r$')
    ax.set_ylabel('Steady State Polarization $q$')
    ax.set_title('Robustness of Phase Transition on Networks')
    ax.legend()
    ax.grid(True)
    
    plt.savefig(OUTPUT_DIR / "Fig3_Network_Validation.png", dpi=300)
    plt.close()
    print("  -> Saved Fig3")

def experiment_4_critical_slowing_down():
    """
    图 4: 临界慢化预警 (Critical Slowing Down)
    随着 r 逼近 rc，自相关系数应该上升。
    """
    print("Running Experiment 4: Critical Slowing Down...")
    
    phi, theta, k_avg = 0.6, 0.4, 50
    n_m, n_w = 10, 5
    chi = theory.calculate_chi(phi, theta, k_avg)
    rc = theory.calculate_rc(n_m, n_w, chi)
    
    # 在临界点左侧（稳定区）逼近
    # 比如 rc=0.6，我们取 0.1, 0.3, 0.5, 0.55
    r_approach = np.linspace(0, rc - 0.02, 10)
    autocorr_values = []
    
    for r in tqdm(r_approach, desc="  Calculating Autocorrelation"):
        alpha, u = theory.get_gl_params(r, rc)
        # alpha > 0，在单底碗里晃动
        # 需要足够长的序列来计算自相关
        cfg = SDEConfig(alpha=float(alpha), u=u, sigma=0.1, dt=0.05, steps=5000)
        _, traj = run_sde_simulation(cfg, q0=0.0) # 从平衡点开始
        
        # 计算 Lag-1 自相关
        ts = traj[:, 0] # 取第一条轨迹
        ac = calculate_autocorrelation(ts, lag=1)
        autocorr_values.append(ac)
        
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.plot(r_approach, autocorr_values, 'o-', color='purple', linewidth=2)
    ax.set_xlabel('Control Parameter $r$ (Approaching Criticality)')
    ax.set_ylabel('Lag-1 Autocorrelation')
    ax.set_title('Early Warning Signal: Critical Slowing Down')
    ax.axvline(rc, color='red', linestyle='--', label=f'Critical Point $r_c$')
    ax.legend()
    ax.grid(True)
    
    plt.savefig(OUTPUT_DIR / "Fig4_CSD.png", dpi=300)
    plt.close()
    print("  -> Saved Fig4")

if __name__ == "__main__":
    print("Starting full experiment suite...")
    experiment_1_bifurcation_diagram()
    experiment_2_potential_landscape()
    experiment_3_network_robustness()
    experiment_4_critical_slowing_down()
    print(f"\nAll figures saved to {OUTPUT_DIR}")

Starting full experiment suite...
Running Experiment 1: Bifurcation Diagram...
  -> Theoretical rc = 0.8233


  SDE Simulation: 100%|██████████| 15/15 [00:01<00:00, 14.51it/s]


  -> Saved Fig1
Running Experiment 2: Potential Landscape...
  -> Saved Fig2
Running Experiment 3: Network Simulation...


  Network ABM: 100%|██████████| 11/11 [00:29<00:00,  2.70s/it]


  -> Saved Fig3
Running Experiment 4: Critical Slowing Down...


  Calculating Autocorrelation: 100%|██████████| 10/10 [00:00<00:00, 15.51it/s]
